In [1]:
from tqdm.notebook import tqdm
from loss.dilate_loss import DilateLoss 
import torch
from pts import Trainer

In [2]:
import os, sys; sys.path.append('..')
from share_src.config import read_config
from share_src.data import make_datasets

from torch.utils.data import DataLoader

cfg = read_config()
train_dataset, test_dataset = make_datasets(cfg)


trainloader = DataLoader(train_dataset, 
    batch_size=cfg.TRAIN.BATCH, shuffle=True, drop_last=True)
testloader = DataLoader(test_dataset, 
    batch_size=cfg.TRAIN.BATCH, shuffle=False, drop_last=True)


DATA:
  NAMES_OF_BAD_FILES: ['23EF07CA2AD4829E5A3DC550DDF212BE4DF432357587117B8790BF98BDB04EF52277040A3F9FB30FFFB875701FFFF0FD_202102230745.xlsx', '490175DD47C1E53DE74F180C750AA30055974E0FEFDF45C5B52C8ADC028C5B7EEC0C441B7C1869130D1B194AD47E4EDD_202103181245.xlsx']
  OVERLAP_LEN: 10
  PATH: ../data/medical_dataset/2nd_version_data
  PREDICT_MAGNITUDE: False
  PROPOFOL_ID: 5
  SOURCE_LEN: 20
  TARGET_LEN: 20
  TEST_PERCENT: 0.2
MODEL:
  ACTIVATION: gelu
  DEC_IN: 11
  DIM_OUT: 1
  DROPOUT: 0.05
  D_FF: 2048
  D_LAYERS: 1
  D_MODEL: 512
  ENC_IN: 11
  E_LAYERS: 2
  FACTOR: 1
  MOV_AVG: 11
  N_HEADS: 8
  POS_ENC_N: 2
SYSTEM:
  SEED: 2022
TEST:
  PREDICTIONS_TO_FILES: True
  SAVE_PATH: results
TRAIN:
  BATCH: 128
  CRITERION: MSE
  DILATE_GAMMA: 0.0001
  EPOCHS: 10
  LR: 0.0001
  USE_ALL_DATA: True
Total number of files read: 628
Number of correct dataframes: 605
TRAIN FILES: 484
TEST FILES: 121
 


In [3]:
from tactis.model.tactis import TACTiS
device = torch.device("cuda")

def create_net():
    net = TACTiS(
        num_series=11,
        series_embedding_dim=5,
        input_encoder_layers=3,
        input_encoding_normalization=True,
        data_normalization="standardization",
        loss_normalization="series",
        positional_encoding={
            "dropout": 0.0,
        },
        temporal_encoder={
            "attention_layers": 3,
            "attention_heads": 3,
            "attention_dim": 16,
            "attention_feedforward_dim": 16,
            "dropout": 0.0,
        },
        copula_decoder={
            "min_u": 0.01,
            "max_u": 0.99,
            "attentional_copula": {
                "attention_heads": 3,
                "attention_layers": 3,
                "attention_dim": 16,
                "mlp_layers": 3,
                "mlp_dim": 16,
                "resolution": 50,
            },
            "dsf_marginal": {
                "mlp_layers": 2,
                "mlp_dim": 8,
                "flow_layers": 2,
                "flow_hid_dim": 8,
            },
        },
    )
    return net.to(device)

In [4]:
model = create_net()

In [5]:
model

TACTiS(
  (series_encoder): Embedding(11, 5)
  (encoder): TemporalEncoder(
    (layer_timesteps): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=48, out_features=48, bias=True)
        )
        (linear1): Linear(in_features=48, out_features=16, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
        (linear2): Linear(in_features=16, out_features=48, bias=True)
        (norm1): LayerNorm((48,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((48,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.0, inplace=False)
        (dropout2): Dropout(p=0.0, inplace=False)
      )
      (1): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=48, out_features=48, bias=True)
        )
        (linear1): Linear(in_features=48, out_features=16, bias=True)
        (drop

In [9]:
N_input = cfg.DATA.SOURCE_LEN
N_output = cfg.DATA.TARGET_LEN  
sigma = 0.01
alpha=0.5
gamma = cfg.TRAIN.DILATE_GAMMA
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


def train_model(net, loss_type, learning_rate, epochs=1000, gamma = 0.001,
                print_every=50,eval_every=50, verbose=1, Lambda=1, alpha=0.5):
    
    optimizer = torch.optim.Adam(net.parameters(),lr=learning_rate)
    criterion_mse = torch.nn.MSELoss()
    criterion_dilate = DilateLoss(alpha, gamma, device)
    
    losses = []
    for epoch in tqdm(range(epochs)): 
        pbar = tqdm(enumerate(trainloader, 0), 
            total=len(trainloader), leave=False)
        for i, data in pbar:
            inputs, target, _ = data
            inputs = torch.tensor(inputs, dtype=torch.float32).to(device)
            target = torch.tensor(target, dtype=torch.float32).to(device)

            # Remove positional data
            pos_inputs = inputs[:,:,-1] + inputs[:,:,-2]
            pos_target = (target[:,cfg.DATA.OVERLAP_LEN:,-1] + 
                          target[:,cfg.DATA.OVERLAP_LEN:,-1])
            inputs = inputs[:,:,:-2]
            target = target[:,cfg.DATA.OVERLAP_LEN:,:-2]
            
            # reshape data to fit model
            batch_size, seq_len, feature_dim = inputs.shape
            inputs = inputs.reshape(batch_size, feature_dim, seq_len)
            target = target.reshape(batch_size, feature_dim, seq_len)
            # pos_inputs = pos_inputs[:,:,1]
            # pos_target = pos_target[:,:,0]

            '''
            inputs/hist_value [batch_size, series, hist_length]
            target/pred_value [batch_size, series, pred_length]
            pos_inputs/hist_time  [batch_size, hist_length]
            pos_target/pred_time  [batch_size, pred_length]
            '''

            optimizer.zero_grad()
            loss = net.loss(pos_inputs, inputs, pos_target, target)
            loss.backward()
            optimizer.step()

            losses += [loss.item()]
            pbar.set_description('loss: {:.4f}'.format(loss.item()))
    
    model = net
    return model, losses

trained_model, losses = train_model(
    model,
    loss_type='dilate',
    learning_rate=cfg.TRAIN.LR,
    epochs=cfg.TRAIN.EPOCHS, 
    gamma=gamma, 
    print_every=50, 
    eval_every=50,
    verbose=1)


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/818 [00:00<?, ?it/s]

/tmp/ipykernel_275243/2670861062.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(inputs, dtype=torch.float32).to(device)
/tmp/ipykernel_275243/2670861062.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target = torch.tensor(target, dtype=torch.float32).to(device)


  0%|          | 0/818 [00:00<?, ?it/s]

  0%|          | 0/818 [00:00<?, ?it/s]

In [39]:
def step(net, optimizer, batch_size, data, hist_length, pred_length):
    max_idx = data.shape[1] - (hist_length + pred_length)
    
    hist_values = []
    pred_values = []
    for _ in range(batch_size):
        idx = np.random.randint(0, max_idx)
        hist_values.append(data[:, idx:idx+hist_length])
        pred_values.append(data[:, idx+hist_length:idx+hist_length+pred_length])
    
    # [batch, series, time steps]
    hist_value = torch.Tensor(hist_values).to(device)
    pred_value = torch.Tensor(pred_values).to(device)
    hist_time = torch.arange(0, hist_length, device=device)[None, :].expand(batch_size, -1)
    pred_time = torch.arange(hist_length, hist_length + pred_length, device=device)[None, :].expand(batch_size, -1)
    
    optimizer.zero_grad()
    loss = net.loss(hist_time, hist_value, pred_time, pred_value)
    loss.backward()
    optimizer.step()
    
    return loss.item()

In [ ]:
net = create_net()
data_train, data_test = generate_random_data()
optimizer = torch.optim.RMSprop(net.parameters(), lr=1e-3, weight_decay=0)

In [ ]:
avg_loss = []

NUM_EPOCHS = 1000  # The model is very slow to train
NUM_BATCHES = 100

for epoch in range(NUM_EPOCHS):
    running_sum = 0
    for batch in range(NUM_BATCHES):
        running_sum += step(net, optimizer, 256, data_train, 10, 10)
    avg_loss.append(running_sum / NUM_BATCHES)

In [ ]:
plt.figure()
plt.plot(avg_loss)

In [ ]:
def sample(net, num_samples, data, hist_length, pred_length):
    max_idx = data.shape[1] - (hist_length + pred_length)
    
    idx = np.random.randint(0, max_idx)
    hist_value = torch.Tensor(data[:, idx:idx+hist_length]).to(device)
    pred_value = torch.Tensor(data[:, idx+hist_length:idx+hist_length+pred_length]).to(device)
    
    # [batch, series, time steps]
    hist_value = hist_value[None, :, :]
    pred_value = pred_value[None, :, :]
    hist_time = torch.arange(0, hist_length, device=device)[None, :]
    pred_time = torch.arange(hist_length, hist_length + pred_length, device=device)[None, :]

    samples = net.sample(num_samples, hist_time, hist_value, pred_time)
    
    return samples, torch.cat([hist_value, pred_value], axis=2), torch.cat([hist_time, pred_time], axis=1)

In [ ]:
samples, pred_value, timesteps = sample(net, 1000, data_test, 10, 10)

In [ ]:
def plot_single_series(samples, target, timesteps, index):
    s_samples = samples[0, index, :, :].cpu().numpy()
    s_timesteps = timesteps[0, :].cpu().numpy()
    s_target = target[0, index, :].cpu().numpy()
    
    plt.figure()
    
    for zorder, quant, color, label in [
        [1, 0.05, (0.75,0.75,1), "5%-95%"],
        [2, 0.10, (0.25,0.25,1), "10%-90%"],
        [3, 0.25, (0,0,0.75), "25%-75%"],
    ]:
        plt.fill_between(
            s_timesteps,
            np.quantile(s_samples, quant, axis=1),
            np.quantile(s_samples, 1 - quant, axis=1),
            facecolor=color,
            interpolate=True,
            label=label,
            zorder=zorder,
        )
    
    plt.plot(
        s_timesteps,
        np.quantile(s_samples, 0.5, axis=1),
        color=(0.5,0.5,0.5),
        linewidth=3,
        label="50%",
        zorder=4,
    )
    
    plt.plot(s_timesteps, s_target, color=(0, 0, 0), linewidth=2, zorder=5, label="ground truth")
    
    handles, labels = plt.gca().get_legend_handles_labels()
    order = [1, 2, 3, 4, 0]
    plt.legend([handles[idx] for idx in order], [labels[idx] for idx in order])
    
    plt.show()

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/195 [00:00<?, ?it/s]

/tmp/ipykernel_140928/4153988318.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(inputs, dtype=torch.float32).to(device)
/tmp/ipykernel_140928/4153988318.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target = torch.tensor(target, dtype=torch.float32).to(device)


NotImplementedError: 

In [1]:
# Configuration
import os
import sys
REPO_NAME = "tactis"
def get_repo_basepath():
    cd = os.path.abspath(os.curdir)
    return cd[:cd.index(REPO_NAME) + len(REPO_NAME)]
REPO_BASE_PATH = get_repo_basepath()
sys.path.append(REPO_BASE_PATH)

In [6]:
from pts import Trainer
import torch
from tactis.gluon.estimator import TACTiSEstimator
from tactis.gluon.dataset import generate_backtesting_datasets
from tactis.gluon.metrics import compute_validation_metrics
from tactis.gluon.plots import plot_four_forecasts
from gluonts.evaluation.backtest import make_evaluation_predictions

In [9]:
predictor = estimator.train(trainloader)

  0%|          | 0/511 [00:00<?, ?it/s]

TypeError: list indices must be integers or slices, not str

In [ ]:
predictor.batch_size = 16
metrics = compute_validation_metrics(
    predictor=predictor,
    dataset=test_data,
    window_length=estimator.history_length + estimator.prediction_length,
    num_samples=100,
    split=False,
)

In [ ]:
metrics

In [ ]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=test_data, predictor=predictor, num_samples=100
)
forecasts = list(forecast_it)
targets = list(ts_it)

In [ ]:
plot_four_forecasts(
    forecasts=forecasts,
    targets=targets,
    selection=[(0, 0), (1, 5), (2, 10), (3, 15)],
    tick_freq="day",
    history_length=estimator.history_length,
)